In [2]:
# ============================================================
# Store Sales – Time Series Forecasting (my end-to-end notebook)
# Michael Ramirez — first-person, production-minded, submission-ready
# ============================================================
#
# What I’m doing here:
# 1) Load + sanity-check all tables from the competition.
# 2) Build a clean date calendar (dow, dom, month, etc.), paydays (15/EOM),
#    earthquake window features, oil price rolls, and holiday flags.
# 3) Safely merge everything, typed categoricals, and create leak-free
#    per-(store_nbr, family) lags/rolling means for `sales`.
# 4) Use a robust sklearn ColumnTransformer (OHE for cats, passthrough nums).
# 5) TimeSeriesSplit CV (RMSLE) to verify the approach.
# 6) Train a final XGBoost model (GPU if available—auto fallback to CPU),
#    generate `submission.csv`, and save minimal artifacts.
#
# Design notes:
# - No deep learning. Classical GBDT with careful feature engineering wins here.
# - No parameter knobs in .fit() that break cross-version (I keep it version-safe).
# - I don’t rely on leakage; all target-based features are lagged/rolled properly.
# - Matplotlib only for visuals (I keep plotting optional/minimal for Kaggle speed).
#
# Tip for Kaggle:
# - Set Accelerator = GPU in the Notebook settings to speed up training.
# - This single notebook produces `submission.csv` in Outputs.
# ============================================================

import os
import json
import math
import warnings
from dataclasses import dataclass, asdict
from typing import Dict, Any, Tuple, List, Optional

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt  # used lightly; safe for headless

from packaging import version

# sklearn
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_log_error

# xgboost
import xgboost as xgb
from xgboost import XGBRegressor

warnings.filterwarnings("ignore")

# ----------------------------
# Reproducibility + Paths
# ----------------------------
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

KAGGLE_DIR = "/kaggle/input/store-sales-time-series-forecasting"
LOCAL_DIR = os.environ.get("DATA_DIR", "")
DATA_DIR = KAGGLE_DIR if os.path.exists(KAGGLE_DIR) else LOCAL_DIR

RUN_ID = pd.Timestamp.now().strftime("%Y%m%d-%H%M%S")
ARTIFACTS_DIR = f"./artifacts_{RUN_ID}"
os.makedirs(ARTIFACTS_DIR, exist_ok=True)

# ----------------------------
# Config (kept flat and explicit)
# ----------------------------
@dataclass
class Config:
    target_col: str = "sales"
    date_col: str = "date"
    id_cols: Tuple[str, str] = ("store_nbr", "family")
    onpromotion_col: str = "onpromotion"

    sales_lags: Tuple[int, ...] = (1, 7, 14, 28)
    sales_rolls: Tuple[int, ...] = (7, 14, 28)

    oil_rolls: Tuple[int, ...] = (7, 14, 28)
    earthquake_date: str = "2016-04-16"
    earthquake_window_days: int = 45
    payday_days: Tuple[int, int] = (15, -1)  # 15th + month-end

    # Model knobs (conservative; I prefer stability over flashy params)
    xgb_params: Dict[str, Any] = None
    n_splits_ts: int = 3
    eval_metric: str = "rmse"

cfg = Config(
    xgb_params=dict(
        n_estimators=2000,
        learning_rate=0.05,
        max_depth=8,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        tree_method="hist",   # will be overridden if GPU is present (2.x -> device='cuda' / 1.x -> gpu_hist)
        n_jobs=-1,
        random_state=RANDOM_STATE,
    )
)

with open(os.path.join(ARTIFACTS_DIR, "config.json"), "w") as f:
    json.dump(asdict(cfg), f, indent=2)


# ----------------------------
# Helpers
# ----------------------------
def mem_usage(df: pd.DataFrame) -> str:
    return f"{df.memory_usage(deep=True).sum() / 1024**2:.2f} MB"

def rmsle(y_true, y_pred) -> float:
    y_true = np.maximum(y_true, 0.0)
    y_pred = np.maximum(y_pred, 0.0)
    return math.sqrt(mean_squared_log_error(y_true, y_pred))


# ----------------------------
# Load Data (dtypes for memory)
# ----------------------------
dtype_train = {
    "store_nbr": "int16",
    "family": "category",
    "onpromotion": "float32",
    "sales": "float32",
}
dtype_test = {
    "store_nbr": "int16",
    "family": "category",
    "onpromotion": "float32",
}

train = pd.read_csv(os.path.join(DATA_DIR, "train.csv"), parse_dates=["date"], dtype=dtype_train)
test  = pd.read_csv(os.path.join(DATA_DIR, "test.csv"),  parse_dates=["date"], dtype=dtype_test)
stores = pd.read_csv(os.path.join(DATA_DIR, "stores.csv"))
oil    = pd.read_csv(os.path.join(DATA_DIR, "oil.csv"), parse_dates=["date"])
hol    = pd.read_csv(os.path.join(DATA_DIR, "holidays_events.csv"), parse_dates=["date"])

print("Train:", train.shape, mem_usage(train))
print("Test :", test.shape, mem_usage(test))


# ----------------------------
# Calendar & Aux Features
# ----------------------------
def add_calendar_parts(cal: pd.DataFrame, date_col: str = "date") -> pd.DataFrame:
    dt = cal[date_col]
    cal["dow"] = dt.dt.weekday.astype("int8")  # Mon=0
    cal["dom"] = dt.dt.day.astype("int8")
    cal["month"] = dt.dt.month.astype("int8")
    cal["year"] = dt.dt.year.astype("int16")
    cal["doy"] = dt.dt.dayofyear.astype("int16")
    cal["wom"] = ((cal["dom"] - 1) // 7 + 1).astype("int8")
    return cal

def add_paydays(cal: pd.DataFrame, date_col="date", days=(15, -1)) -> pd.DataFrame:
    d = cal.copy()
    d["is_payday_15"] = (d[date_col].dt.day == days[0]).astype("int8")
    month_end = (d[date_col] + pd.offsets.MonthEnd(0)).dt.day
    d["is_payday_eom"] = (d[date_col].dt.day == month_end).astype("int8")
    d["days_to_15"] = (d[date_col].dt.day - days[0]).abs().clip(0, 10).astype("int8")
    d["days_to_eom"] = (month_end - d[date_col].dt.day).abs().clip(0, 10).astype("int8")
    return d

def add_earthquake(cal: pd.DataFrame, quake_date: str, window_days: int) -> pd.DataFrame:
    d = cal.copy()
    qd = pd.to_datetime(quake_date)
    delta = (d["date"] - qd).dt.days
    d["days_since_quake"] = delta.clip(lower=-365, upper=365).astype("int16")
    d["is_quake_window"] = ((delta >= 0) & (delta <= window_days)).astype("int8")
    return d

def prepare_oil(oil_df: pd.DataFrame) -> pd.DataFrame:
    o = oil_df.rename(columns={"dcoilwtico": "oil_price"}).copy()
    full_range = pd.date_range(o["date"].min(), o["date"].max(), freq="D")
    o = o.set_index("date").reindex(full_range).rename_axis("date").reset_index()
    o["oil_price"] = o["oil_price"].ffill()
    return o

def add_oil_rolls(cal: pd.DataFrame, oil_df: pd.DataFrame, rolls=(7, 14, 28)) -> pd.DataFrame:
    o = prepare_oil(oil_df)
    c = cal.merge(o, on="date", how="left")
    for w in rolls:
        c[f"oil_rollmean_{w}"] = c["oil_price"].rolling(w, min_periods=max(2, w//2)).mean()
    return c

def prepare_holidays(hol: pd.DataFrame) -> pd.DataFrame:
    h = hol.copy()
    h["is_holiday_base"] = ((h["type"].isin(["Holiday", "Transfer"])) & (h["transferred"] == False)).astype("int8")
    h["is_transferred_from"] = ((h["type"] == "Holiday") & (h["transferred"] == True)).astype("int8")
    h["is_bridge"] = (h["type"] == "Bridge").astype("int8")
    h["is_work_day"] = (h["type"] == "Work Day").astype("int8")
    h["is_additional"] = (h["type"] == "Additional").astype("int8")
    agg = h.groupby("date", as_index=False)[["is_holiday_base","is_transferred_from","is_bridge","is_work_day","is_additional"]].max()
    agg["is_holiday"] = ((agg["is_holiday_base"] == 1) | (agg["is_transferred_from"] == 1)).astype("int8")
    return agg

mind = min(train["date"].min(), test["date"].min())
maxd = max(train["date"].max(), test["date"].max())
calendar = pd.DataFrame({"date": pd.date_range(mind, maxd, freq="D")})
calendar = add_calendar_parts(calendar)
calendar = add_paydays(calendar, "date", cfg.payday_days)
calendar = add_earthquake(calendar, cfg.earthquake_date, cfg.earthquake_window_days)
calendar = add_oil_rolls(calendar, oil, cfg.oil_rolls)
hol_agg = prepare_holidays(hol)
calendar = calendar.merge(hol_agg, on="date", how="left")
calendar[["is_holiday","is_bridge","is_work_day","is_additional"]] = \
    calendar[["is_holiday","is_bridge","is_work_day","is_additional"]].fillna(0).astype("int8")
calendar["oil_price"] = calendar["oil_price"].ffill()

# ----------------------------
# Merge store metadata + calendar
# ----------------------------
stores_renamed = stores.rename(columns={"type": "store_type"}).copy()

for df in (train, test):
    df["onpromotion"] = df["onpromotion"].fillna(0).astype("float32")

train = train.merge(stores_renamed, on="store_nbr", how="left")
test  = test.merge(stores_renamed, on="store_nbr", how="left")

train = train.merge(calendar, on="date", how="left")
test  = test.merge(calendar, on="date", how="left")

for c in ["city", "state", "store_type"]:
    train[c] = train[c].astype("category")
    test[c]  = test[c].astype("category")

# ----------------------------
# Leak-free lags/rolls by (store_nbr, family)
# ----------------------------
def add_group_lag_roll(df_all: pd.DataFrame, cfg: Config) -> pd.DataFrame:
    df = df_all.sort_values(["store_nbr", "family", "date"]).copy()
    g = ["store_nbr", "family"]
    for lag in cfg.sales_lags:
        df[f"sales_lag_{lag}"] = df.groupby(g)["sales"].shift(lag)
    for w in cfg.sales_rolls:
        df[f"sales_rollmean_{w}"] = (
            df.groupby(g)["sales"].apply(lambda s: s.shift(1).rolling(window=w, min_periods=max(2, w//2)).mean())
              .reset_index(level=g, drop=True)
        )
    return df

test_with_sales = test.copy()
test_with_sales["sales"] = np.nan
all_df = pd.concat([train, test_with_sales], axis=0, ignore_index=True)

all_df = add_group_lag_roll(all_df, cfg)

train = all_df[all_df["sales"].notna()].copy()
test  = all_df[all_df["sales"].isna()].copy().drop(columns=["sales"])

lag_roll_cols = [c for c in train.columns if c.startswith("sales_lag_") or c.startswith("sales_rollmean_")]
for c in lag_roll_cols:
    train[c] = train[c].fillna(0.0).astype("float32")
    test[c]  = test[c].fillna(0.0).astype("float32")

# ----------------------------
# Feature matrix
# ----------------------------
TARGET = cfg.target_col

categorical_cols = ["family","city","state","store_type","cluster"]
train["cluster"] = train["cluster"].astype(str).astype("category")
test["cluster"]  = test["cluster"].astype(str).astype("category")

numeric_cols = [
    "store_nbr","onpromotion",
    "dow","dom","month","year","doy","wom",
    "is_payday_15","is_payday_eom","days_to_15","days_to_eom",
    "days_since_quake","is_quake_window",
    "oil_price","oil_rollmean_7","oil_rollmean_14","oil_rollmean_28",
    "is_holiday","is_bridge","is_work_day","is_additional",
] + lag_roll_cols

feature_cols = categorical_cols + numeric_cols

X = train[feature_cols].copy()
y = train[TARGET].astype("float32").values

# sklearn version-safe OHE arg
import sklearn
if version.parse(sklearn.__version__) >= version.parse("1.2"):
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True)
else:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=True)

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", ohe, categorical_cols),
        ("num", "passthrough", numeric_cols),
    ],
    remainder="drop"
)

# ----------------------------
# TimeSeriesSplit CV (RMSLE)
# ----------------------------
train_sorted_idx = np.argsort(train["date"].values)
X_sorted = X.iloc[train_sorted_idx]
y_sorted = y[train_sorted_idx]

tscv = TimeSeriesSplit(n_splits=cfg.n_splits_ts)

# XGBoost GPU handling (version-safe)
gpu_params = {}
if version.parse(xgb.__version__) >= version.parse("2.0.0"):
    gpu_params.update({"device": "cuda"})  # 2.x
else:
    gpu_params.update({"tree_method": "gpu_hist", "predictor": "gpu_predictor"})  # 1.x

def build_model_params(base: dict) -> dict:
    mp = dict(base) if base is not None else {}
    mp.update(gpu_params)
    mp.setdefault("verbosity", 0)
    mp.setdefault("eval_metric", cfg.eval_metric)
    return mp

fold_scores = []
feature_names_out = None

for fold, (tr_idx, va_idx) in enumerate(tscv.split(X_sorted), 1):
    X_tr_raw, X_va_raw = X_sorted.iloc[tr_idx], X_sorted.iloc[va_idx]
    y_tr, y_va = y_sorted[tr_idx], y_sorted[va_idx]

    Xt = preprocessor.fit_transform(X_tr_raw)
    Xv = preprocessor.transform(X_va_raw)

    params = build_model_params(cfg.xgb_params)
    try:
        model = XGBRegressor(**params)
        model.fit(Xt, y_tr, eval_set=[(Xv, y_va)])
    except xgb.core.XGBoostError as e:
        print("[WARN] GPU failed; falling back to CPU ->", str(e))
        params.pop("device", None)
        params["tree_method"] = "hist"
        params["predictor"] = "auto"
        model = XGBRegressor(**params)
        model.fit(Xt, y_tr, eval_set=[(Xv, y_va)])

    va_pred = np.maximum(model.predict(Xv), 0.0)
    score = rmsle(y_va, va_pred)
    fold_scores.append(score)
    print(f"Fold {fold}: RMSLE={score:.5f}")

    if feature_names_out is None:
        try:
            cat_features = list(preprocessor.named_transformers_["cat"].get_feature_names_out(categorical_cols))
            feature_names_out = cat_features + numeric_cols
        except Exception:
            feature_names_out = None

print("CV RMSLE:", fold_scores, "| Mean:", float(np.mean(fold_scores)))
with open(os.path.join(ARTIFACTS_DIR, "cv_metrics.json"), "w") as f:
    json.dump({"fold_rmsle": [float(s) for s in fold_scores],
               "mean_rmsle": float(np.mean(fold_scores))}, f, indent=2)

# ----------------------------
# Final fit on (train + last val chunk), predict test, write submission
# ----------------------------
last_tr_idx, last_va_idx = list(tscv.split(X_sorted))[-1]
X_tr_raw = X_sorted.iloc[last_tr_idx]; X_va_raw = X_sorted.iloc[last_va_idx]
y_tr = y_sorted[last_tr_idx]; y_va = y_sorted[last_va_idx]

# fresh preprocessor for deployment
preprocessor_full = ColumnTransformer(
    transformers=[
        ("cat", ohe, categorical_cols),
        ("num", "passthrough", numeric_cols),
    ],
    remainder="drop"
)

X_tr_all = pd.concat([X_tr_raw, X_va_raw], axis=0)
y_tr_all = np.concatenate([y_tr, y_va], axis=0)

X_tr_all_t = preprocessor_full.fit_transform(X_tr_all)
X_val_t    = preprocessor_full.transform(X_va_raw)

params = build_model_params(cfg.xgb_params)
try:
    model_full = XGBRegressor(**params)
    model_full.fit(X_tr_all_t, y_tr_all, eval_set=[(X_val_t, y_va)])
except xgb.core.XGBoostError as e:
    print("[WARN] GPU failed; falling back to CPU ->", str(e))
    params.pop("device", None)
    params["tree_method"] = "hist"
    params["predictor"] = "auto"
    model_full = XGBRegressor(**params)
    model_full.fit(X_tr_all_t, y_tr_all, eval_set=[(X_val_t, y_va)])

# test transform + predict
X_test = test[feature_cols].copy()
X_test_t = preprocessor_full.transform(X_test)
test_pred = np.maximum(model_full.predict(X_test_t), 0.0)

# submission
sample = pd.read_csv(os.path.join(DATA_DIR, "sample_submission.csv"))
submission = sample.copy()
submission["sales"] = test_pred
sub_path = os.path.join(ARTIFACTS_DIR, f"submission_{RUN_ID}.csv")
submission.to_csv(sub_path, index=False)
print("Wrote:", sub_path)

# Save minimal artifacts (handy for debugging offline)
import joblib
joblib.dump(model_full, os.path.join(ARTIFACTS_DIR, "model_xgb.joblib"))
joblib.dump(preprocessor_full, os.path.join(ARTIFACTS_DIR, "preprocessor.joblib"))
with open(os.path.join(ARTIFACTS_DIR, "feature_names.json"), "w") as f:
    json.dump(feature_names_out, f, indent=2)

# Also emit a copy named exactly 'submission.csv' for Kaggle auto-pickup in Outputs
submission.to_csv("submission.csv", index=False)
print("submission.csv saved (ready to Submit to Competition)")


Train: (3000888, 6) 77.27 MB
Test : (28512, 5) 0.63 MB
[0]	validation_0-rmse:937.34750
[1]	validation_0-rmse:895.40563
[2]	validation_0-rmse:856.17577
[3]	validation_0-rmse:818.70397
[4]	validation_0-rmse:783.53362
[5]	validation_0-rmse:750.03482
[6]	validation_0-rmse:718.26120
[7]	validation_0-rmse:688.04253
[8]	validation_0-rmse:659.63321
[9]	validation_0-rmse:633.25712
[10]	validation_0-rmse:607.97543
[11]	validation_0-rmse:584.19573
[12]	validation_0-rmse:561.28865
[13]	validation_0-rmse:540.52994
[14]	validation_0-rmse:520.52535
[15]	validation_0-rmse:502.20271
[16]	validation_0-rmse:484.09453
[17]	validation_0-rmse:465.05128
[18]	validation_0-rmse:449.56676
[19]	validation_0-rmse:434.53053
[20]	validation_0-rmse:420.33526
[21]	validation_0-rmse:407.49145
[22]	validation_0-rmse:395.47605
[23]	validation_0-rmse:383.87620
[24]	validation_0-rmse:373.40752
[25]	validation_0-rmse:363.36987
[26]	validation_0-rmse:353.97553
[27]	validation_0-rmse:345.29632
[28]	validation_0-rmse:335.4751